In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport

# Inspección de variables
Cargar los documentos y revisar variables categóricas y numericas

In [3]:
df_pump = pd.read_csv('training_values.csv', delimiter=',')
df_label = pd.read_csv('training_labels.csv', delimiter=',')

print(df_pump.shape)
print(df_label.shape)

(59400, 40)
(59400, 2)


In [4]:
print(df_pump['id'].duplicated().value_counts())
print(df_label['id'].duplicated().value_counts())

id
False    59400
Name: count, dtype: int64
id
False    59400
Name: count, dtype: int64


In [9]:
print(df_label['id'].isin(df_pump['id']).value_counts())

id
True    59400
Name: count, dtype: int64


In [5]:
var_categoricas = df_pump.select_dtypes(include=['object','category']).columns
var_num = df_pump.select_dtypes(exclude=['object','category']).columns

In [6]:
df_pump.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [12]:
pr = ProfileReport(df=df_pump)
pr.to_file('reporte.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
print(var_categoricas)
print(var_num)

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')
Index(['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude',
       'num_private', 'region_code', 'district_code', 'population',
       'construction_year'],
      dtype='object')


In [ ]:
df_pump['subvillage'].value_counts(dropna=False, normalize=False).where(lambda x: x > 100).dropna().sort_values(ascending=False)
# .where(lambda x: x > 500).dropna().sort_values(ascending=False)

subvillage
Madukani      508.0
Shuleni       506.0
Majengo       502.0
Kati          373.0
NaN           371.0
Mtakuja       262.0
Sokoni        232.0
M             187.0
Muungano      172.0
Mbuyuni       164.0
Mlimani       152.0
Songambele    147.0
Msikitini     134.0
Miembeni      134.0
1             132.0
Kibaoni       114.0
Kanisani      111.0
I             109.0
Mapinduzi     109.0
Mjimwema      108.0
Mjini         108.0
Mkwajuni      104.0
Mwenge        102.0
Name: count, dtype: float64

In [83]:
field_to_check='installer'
df_pump[df_pump[field_to_check].str.len() <= 2][field_to_check].value_counts(dropna=False, normalize=False)

installer
0     777
WU    301
DW    246
Da    224
DH    202
     ... 
MW      1
R       1
Nu      1
WA      1
M       1
Name: count, Length: 109, dtype: int64

In [81]:
df_pump[df_pump['installer'] == 'RWE'][['funder', 'installer', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name','management',
       'management_group']]

,funder,installer,public_meeting,recorded_by,scheme_management,scheme_name,management,management_group
62,Norad,RWE,True,GeoData Consultants Ltd,VWC,Mkongoro One,vwc,user-group
70,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,NaN,NaN,vwc,user-group
77,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,VWC,Tengeru gravity water supply,vwc,user-group
143,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,VWC,Vugiro,vwc,user-group
173,Dhv,RWE,True,GeoData Consultants Ltd,NaN,NaN,vwc,user-group
...,...,...,...,...,...,...,...,...
59088,World Bank,RWE,True,GeoData Consultants Ltd,VWC,Mnyuzi water supply,vwc,user-group
59104,W.D & I.,RWE,True,GeoData Consultants Ltd,VWC,Mkata ri,vwc,user-group
59205,France,RWE,True,GeoData Consultants Ltd,VWC,Tangeni,vwc,user-group
59269,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,VWC,Muriti Water Supply,vwc,user-group


Trabajar con la variable tipo Date - que se puede extraer rapidamente valores numericos

In [ ]:
df_pump['date_recorded'] = pd.to_datetime(df_pump['date_recorded'])
df_pump['year'] = df_pump['date_recorded'].dt.year
df_pump['month'] = df_pump['date_recorded'].dt.month
df_pump['day'] = df_pump['date_recorded'].dt.day
df_pump['day_of_week'] = df_pump['date_recorded'].dt.dayofweek
df_pump['day_of_week'].value_counts(normalize=True)


day_of_week
2    0.156330
4    0.148434
3    0.148401
1    0.143131
5    0.139646
0    0.139562
6    0.124495
Name: proportion, dtype: float64

In [26]:
df_pump['duracion']=(pd.to_datetime('2013-12-31') - df_pump['date_recorded']).dt.days

In [ ]:
#Separa actividades que son realizadas por personas o entidades
df_pump[['funder', 'installer', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name','management',
       'management_group']]

,funder,installer,wpt_name,public_meeting,recorded_by,scheme_management,scheme_name,management,management_group
0,Roman,Roman,none,True,GeoData Consultants Ltd,VWC,Roman,vwc,user-group
1,Grumeti,GRUMETI,Zahanati,NaN,GeoData Consultants Ltd,Other,NaN,wug,user-group
2,Lottery Club,World vision,Kwa Mahundi,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,vwc,user-group
3,Unicef,UNICEF,Zahanati Ya Nanyumbu,True,GeoData Consultants Ltd,VWC,NaN,vwc,user-group
4,Action In A,Artisan,Shuleni,True,GeoData Consultants Ltd,NaN,NaN,other,other
...,...,...,...,...,...,...,...,...,...
59395,Germany Republi,CES,Area Three Namba 27,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,water board,user-group
59396,Cefa-njombe,Cefa,Kwa Yahona Kuvala,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,vwc,user-group
59397,NaN,NaN,Mashine,True,GeoData Consultants Ltd,VWC,NaN,vwc,user-group
59398,Malec,Musa,Mshoro,True,GeoData Consultants Ltd,VWC,NaN,vwc,user-group


In [50]:
# Separado para analizar con variables que tienen que ver con zonas geográficas
df_pump[['wpt_name',
'basin',
'subvillage',
'region',
'lga',
'ward']]

,wpt_name,basin,subvillage,region,lga,ward
0,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi
1,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta
2,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika
3,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu
4,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi
...,...,...,...,...,...,...
59395,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi
59396,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo
59397,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala
59398,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu


date_recorded = fecha -> calcular dia, mes, año, duracion
funder - 1986 distintos
installer - 2145 d
wpt_name
basil
subvillage
region
lga
ward
public_meeting
recorded_by - unico valor - REMOVER
scheme_management
scheme_name
permit
extraction_type
extraction_type_group
extraction_type_class
management
management_group
payment
payment_type
water_quality
quality_group
quantity
quantity_group
source
source_type
source_class
waterpoint_type
waterpoint_type_group